<a href="https://colab.research.google.com/github/Wisdom-Nji/research.google-projects/blob/master/Word_Vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install spacy
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051304 sha256=f31e31305789139d9d807a5cb58ed55c724af246ae0297df7d0022c0cedf5aa5
  Stored in directory: /tmp/pip-ephem-wheel-cache-6esfcrg6/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [1]:
import spacy

nlp = spacy.load("en_core_web_md")

In [2]:
class Category:
  URGENT = "URGENT" 
  RESULTS = "RESULT"
  QUESTIONS = "QUESTION"
  STATEMENTS = "STATEMENT"


In [3]:
# Data = { URGENT or RESULT}
# Upload Question from data files, which allow for more data to be added to the entire mix
train_x = ['The test was positive', 'Nobody got sick', 'We do not have TB', 'You results are available', "I'm not sick", "I am not negative"]
train_x1 = ['Please send help', 'I need someone right now', 'Her water just broke', "i've been shot"]
train_y = [Category.RESULTS, Category.RESULTS, Category.RESULTS, Category.RESULTS, Category.RESULTS, Category.RESULTS]
train_y1 = [Category.URGENT, Category.URGENT, Category.URGENT, Category.URGENT]

train_x += train_x1 
train_y += train_y1
print( len(train_y), ":", train_y )

10 : ['RESULT', 'RESULT', 'RESULT', 'RESULT', 'RESULT', 'RESULT', 'URGENT', 'URGENT', 'URGENT', 'URGENT']


In [4]:
# Data = {Question or Statement}
train_questions = ['What drug is this?', 'would I have to pay?', 'is this free?', 'can this not be for free?', 'should i come back later?', 'am I tired?', "could we know for sure?", "should I ask?", "could I have TB?"]
train_label_statements = [Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS]
train_label_statements1 = [Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS]
train_label_questions = [Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS]

train_sq_data = train_label_statements + train_label_statements1 + train_label_questions

train_cat_data = []
train_cat_data += train_x + train_questions

In [5]:
# training cat data
docs = [nlp(text) for text in train_x]
train_x_word_vectors = [x.vector for x in docs]

In [6]:
# training sq data
docs = [nlp(text) for text in train_cat_data]
train_x_word_vectors_for_questions = [x.vector for x in docs]

In [7]:
from sklearn import svm
clf_svm_wv = svm.SVC(kernel='linear')
clf_svm_wv.fit(train_x_word_vectors, train_y)

# Training for Questions or statements
clf_svm_wv_sq = svm.SVC(kernel='linear')
clf_svm_wv_sq.fit(train_x_word_vectors_for_questions, train_sq_data)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
import pickle

cat_fit_dump = pickle.dumps( clf_svm_wv)
print( cat_fit_dump )

b'\x80\x03csklearn.svm._classes\nSVC\nq\x00)\x81q\x01}q\x02(X\x17\x00\x00\x00decision_function_shapeq\x03X\x03\x00\x00\x00ovrq\x04X\n\x00\x00\x00break_tiesq\x05\x89X\x06\x00\x00\x00kernelq\x06X\x06\x00\x00\x00linearq\x07X\x06\x00\x00\x00degreeq\x08K\x03X\x05\x00\x00\x00gammaq\tX\x05\x00\x00\x00scaleq\nX\x05\x00\x00\x00coef0q\x0bG\x00\x00\x00\x00\x00\x00\x00\x00X\x03\x00\x00\x00tolq\x0cG?PbM\xd2\xf1\xa9\xfcX\x01\x00\x00\x00Cq\rG?\xf0\x00\x00\x00\x00\x00\x00X\x02\x00\x00\x00nuq\x0eG\x00\x00\x00\x00\x00\x00\x00\x00X\x07\x00\x00\x00epsilonq\x0fG\x00\x00\x00\x00\x00\x00\x00\x00X\t\x00\x00\x00shrinkingq\x10\x88X\x0b\x00\x00\x00probabilityq\x11\x89X\n\x00\x00\x00cache_sizeq\x12K\xc8X\x0c\x00\x00\x00class_weightq\x13NX\x07\x00\x00\x00verboseq\x14\x89X\x08\x00\x00\x00max_iterq\x15J\xff\xff\xff\xffX\x0c\x00\x00\x00random_stateq\x16NX\x07\x00\x00\x00_sparseq\x17\x89X\r\x00\x00\x00class_weight_q\x18cnumpy.core.multiarray\n_reconstruct\nq\x19cnumpy\nndarray\nq\x1aK\x00\x85q\x1bC\x01bq\x1c\x87q\x1dR

In [12]:
test_x = ["Can I get tested?"]
test_docs = [nlp(text) for text in test_x]

test_x_word_vectors = [x.vector for x in test_docs]

print("Statement|Question:", clf_svm_wv_sq.predict( test_x_word_vectors ))
print("Urgent|Results:", clf_svm_wv.predict( test_x_word_vectors ))

Statement|Question: ['QUESTION']
Urgent|Results: ['URGENT']
